In [1]:
# Install Python libraries
print("Installing Python libraries...\n")
!pip install plyfile Pillow==9.5.0 pycolmap mediapy

# Install system packages required for COLMAP
print("\nInstalling system packages for COLMAP...\n")
!sudo apt-get update -y
!sudo apt-get install -y libglm-dev libeigen3-dev libflann-dev libsqlite3-dev \
    libgl1-mesa-dev cmake libgoogle-glog-dev libgflags-dev libsuitesparse-dev \
    libceres-dev libcgal-dev libcgal-qt5-dev libmetis-dev libfreeimage-dev libglew-dev

# Install other utilities
print("\nInstalling other utilities...\n")
!apt install -y imagemagick
!sed -i '88d' ~/../etc/ImageMagick-6/policy.xml  # Remove a specific line to run MoviePy

# Clone and build COLMAP from the official repository
print("\nCloning and building COLMAP...\n")
!git clone https://github.com/colmap/colmap.git
%cd colmap
!mkdir -p build
%cd build
!cmake .. -DCMAKE_CUDA_ARCHITECTURES=native -GNinja
!ninja
!sudo ninja install

# Verify that COLMAP is installed
print("\nVerifying COLMAP installation...\n")
!colmap -h

# Navigate back to the root directory
%cd /content

# Clone HLOC and install its dependencies
print("\nCloning HLOC and installing dependencies...\n")
!git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/
%cd Hierarchical-Localization
!pip install --quiet -e .
!pip install --quiet --upgrade plotly

%cd /content
print("\nInstallation completed!")

/notebooks
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://deb.nodesource.com/node_16.x focal InRelease                     
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease      
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]   
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 1s (393 kB/s)  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglm-dev is already the newest version (0.9.9.7+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 157 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
imagemagick is already the newest version (8:6.9.10.23+dfsg-2

In [3]:
import glob
import os
import sys
import shutil
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapy as media
from pathlib import Path
from time import time
import torch
import pycolmap

%cd /content/Hierarchical-Localization
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval, colmap_from_nvm, pairs_from_exhaustive, pairs_from_covisibility
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

import sys
sys.path.append('/notebooks/Hierarchical-Localization/third_party') 
sys.path.append('/notebooks/Hierarchical-Localization/third_party/d2net')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/deep-image-retrieval')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/r2d2')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/SuperGluePretrainedNetwork')
%cd /content

/notebooks/Hierarchical-Localization
/notebooks
/notebooks


In [ ]:
%%time
%cd /content

# Prepare directories and input images
print("Preparing directories and extracting frames from video...\n")

# Creating directories
work_dir = "/content/3DReconstruction"
input_dir = f"{work_dir}/frames"
output_dir = f"{work_dir}/reconstruction_output"

!mkdir -p {input_dir}
!mkdir -p {output_dir}

# Extracting frames from the video
video_files = sorted(glob.glob("/content/data/*.mp4"))
video_file = video_files[1]
!ffmpeg -i {video_file} -vf fps=15 {input_dir}/%05d.png

# Displaying extracted frames
extracted_frames = sorted(glob.glob(f"{input_dir}/*.png"))
print("Number of images:", len(extracted_frames))

# Display the first 16 frames
show_images = [cv2.cvtColor(cv2.imread(frame), cv2.COLOR_BGR2RGB) for frame in extracted_frames[:16]]
media.show_images(show_images, height=300, columns=4)

In [ ]:

%%time
print("\nPerforming Structure from Motion (SfM) and 3D reconstruction...\n")

# Paths for the reconstruction process
sfm_dir = f"{work_dir}/sfm"
sfm_pairs = f"{sfm_dir}/pairs-sfm.txt"
features_file = f"{sfm_dir}/features.h5"
matches_file = f"{sfm_dir}/matches.h5"

# Creating the sfm directory
!mkdir -p {sfm_dir}

# Extracting features, generating pairs and matching features
extract_features.main(feature_conf, input_dir, image_list=image_list, feature_path=features_file)
pairs_from_exhaustive.main(sfm_pairs, image_list=image_list)
match_features.main(matcher_conf, sfm_pairs, features=features_file, matches=matches_file)

# Performing 3D reconstruction
model = reconstruction.main(sfm_dir, input_dir, sfm_pairs, features_file, matches_file, image_list=image_list)

# Visualizing 3D reconstruction
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

In [ ]:
%%time
print("\nRunning COLMAP image undistorter...\n")

# Running COLMAP image undistorter
undistorted_path = f"{work_dir}/undistorted"
!colmap image_undistorter \
	--image_path {input_dir} \
	--input_path {sfm_dir} \
	--output_path {undistorted_path} \
	--output_type COLMAP

# Organizing the sparse folder
!mv {undistorted_path}/sparse/ {undistorted_path}/sparse_tmp/
!mkdir {undistorted_path}/sparse/
!mv {undistorted_path}/sparse_tmp/ {undistorted_path}/sparse/0/

In [ ]:
# Running Gaussian splatting training
%cd /content
!python /content/gaussian-splatting/train.py -s {undistorted_path} -r 2

print("\nProcess completed!")
